In [1]:
import sys
sys.path.append("../src")

In [8]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import statsmodels.api as sm

import urllib.parse
import requests
import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Project Bond King\Curvy-CUSIPs\notebooks\../.env
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [10]:
date = datetime(2024, 10, 31)
sdr = curve_data_fetcher.dtcc_sdr_fetcher.fetch_dtcc_sdr_data_timeseries(
    start_date=date , end_date=date, agency="CFTC", asset_class="RATES"
)
sdr_df = sdr[date]

FETCHING DTCC SDR DATASETS...: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


Vanilla Receiver Swaptions UPI FISNs: 
- "NA/O Call Epn OIS USD" 
- "NA/O Call Epn Fxd Flt USD"
- "NA/O Call Epn USD"

Vanilla Payer Swaptions UPI FISNs:
- "NA/O P Epn OIS USD"
- "NA/O P Epn Fxd Flt USD"
- "NA/O P Epn USD"

In [38]:
swaption_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_swaption_upis.csv")
fixed_float_swaps_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_fixed_float_swaps_upis.csv")

In [39]:
swaption_dsb_lookup_df

,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,Derived_ClassificationType,...,Derived_UnderlierName,Derived_UnderlyingAssetType,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_UnderlyingInstrumentUPI,Attributes_NotionalCurrency,Attributes_OptionType,Attributes_OptionExerciseStyle,Attributes_ValuationMethodorTrigger,Attributes_DeliveryType
0,1,Rates,Option,Swaption,UPI,QZC0CLTC9GLV,New,NaN,2024-11-06T06:19:26,HRMDVP,...,NA/Rts Swaps Oth GBP,Other,European-Put,Physical,QZ41LPV34PNB,GBP,PUTO,EURO,Vanilla,PHYS
1,1,Rates,Option,Swaption,UPI,QZSMK59MCJNG,New,NaN,2024-11-05T15:01:24,HRMAVC,...,NA/Rts Swaps Oth EUR,Other,European-Call,Cash,QZVNXWJJWZBG,EUR,CALL,EURO,Vanilla,CASH
2,1,Rates,Option,Swaption,UPI,QZ5L36XD0R3W,New,NaN,2024-11-05T14:06:10,HRDFMC,...,NA/Swap Fxd Fxd EUR ILS,Fixed - Fixed,Bermudan-Put,Cash,QZH9PMZQKRKP,ILS,PUTO,BERM,Other,CASH
3,1,Rates,Option,Swaption,UPI,QZ5G2ZX4K79K,New,NaN,2024-11-05T09:21:48,HRHDVC,...,NA/Swap OIS JPY,Overnight Index Swap (OIS),European-Put,Cash,QZNFBFS2TF2G,JPY,PUTO,EURO,Vanilla,CASH
4,1,Rates,Option,Swaption,UPI,QZRBK39CPSH8,New,NaN,2024-11-05T03:50:33,HRCDVP,...,NA/Swap Fxd Flt JPY,Fixed - Floating,European-Put,Physical,QZDKCCTLTCD3,EUR,PUTO,EURO,Vanilla,PHYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5862,1,Rates,Option,Swaption,UPI,QZ6T8T80PL77,New,NaN,2023-11-13T22:48:21,HRMAVC,...,NA/Swap Zero Cpn EUR,Other,European-Call,Cash,QZPPCC95PGW5,EUR,CALL,EURO,Vanilla,CASH
5863,1,Rates,Option,Swaption,UPI,QZWW2XV7MQPS,New,NaN,2023-11-13T22:47:06,HRCCVP,...,NA/Swap Fxd Flt KRW,Fixed - Floating,Bermudan-Call,Physical,QZXJG6223JH7,KRW,CALL,BERM,Vanilla,PHYS
5864,1,Rates,Option,Swaption,UPI,QZXT8WG9GN6T,New,NaN,2023-11-13T22:42:21,HRCHVC,...,NA/Swap Fxd Flt USD,Fixed - Floating,American-Chooser,Cash,QZ26PGG2Q456,USD,OPTL,AMER,Vanilla,CASH
5865,1,Rates,Option,Swaption,UPI,QZ2N8XFRZFW4,New,NaN,2023-11-13T22:42:20,HRCFVC,...,NA/Swap Fxd Flt EUR,Fixed - Floating,Bermudan-Put,Cash,QZ9B21WPR0SH,EUR,PUTO,BERM,Vanilla,CASH


In [24]:
usd_sofr_ois_lookup_conds = (
    (fixed_float_swaps_dsb_lookup_df["Header_UseCase"] == "Fixed_Float_OIS") &
    (fixed_float_swaps_dsb_lookup_df["Derived_UnderlyingAssetType"] == "Overnight Index Swap (OIS)") &
    (fixed_float_swaps_dsb_lookup_df["Derived_ShortName"] == "NA/Swap OIS USD") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_NotionalCurrency"] == "USD") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_NotionalSchedule"] == "Constant") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_DeliveryType"] == "PHYS") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_ReferenceRateTermValue"] == 1) &
    (fixed_float_swaps_dsb_lookup_df["Attributes_ReferenceRateTermUnit"] == "DAYS") &
    (fixed_float_swaps_dsb_lookup_df["Derived_UnderlierName"] == "USD-SOFR-OIS Compound")   
)

underlying_row = fixed_float_swaps_dsb_lookup_df[usd_sofr_ois_lookup_conds]
assert len(underlying_row) == 1
underlying_dict = underlying_row.to_dict("records")[0]
underlying_dict

{'TemplateVersion': 1,
 'Header_AssetClass': 'Rates',
 'Header_InstrumentType': 'Swap',
 'Header_UseCase': 'Fixed_Float_OIS',
 'Header_Level': 'UPI',
 'Identifier_UPI': 'QZPB5VSBGRCD',
 'Identifier_Status': 'New',
 'Identifier_StatusReason': nan,
 'Identifier_LastUpdateDateTime': '2023-10-15T13:20:35',
 'Derived_ClassificationType': 'SRHCSP',
 'Derived_UnderlyingAssetType': 'Overnight Index Swap (OIS)',
 'Derived_SingleorMultiCurrency': 'Single Currency',
 'Derived_CFIDeliveryType': 'Physical',
 'Derived_ShortName': 'NA/Swap OIS USD',
 'Derived_UnderlierName': 'USD-SOFR-OIS Compound',
 'Attributes_NotionalCurrency': 'USD',
 'Attributes_ReferenceRate': 'USD-SOFR-OIS Compound',
 'Attributes_ReferenceRateTermValue': 1,
 'Attributes_ReferenceRateTermUnit': 'DAYS',
 'Attributes_NotionalSchedule': 'Constant',
 'Attributes_DeliveryType': 'PHYS'}

In [27]:
usd_sofr_ois_epn_receiver_conds = (
    (swaption_dsb_lookup_df["Attributes_UnderlyingInstrumentUPI"] == underlying_dict["Identifier_UPI"])
)

swaption_dsb_lookup_df[usd_sofr_ois_epn_receiver_conds]

,Unnamed: 0,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,...,Derived_UnderlierName,Derived_UnderlyingAssetType,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_UnderlyingInstrumentUPI,Attributes_NotionalCurrency,Attributes_OptionType,Attributes_OptionExerciseStyle,Attributes_ValuationMethodorTrigger,Attributes_DeliveryType
6,6,1,Rates,Option,Swaption,UPI,QZDPTZ5FJWW8,New,NaN,2024-11-07T15:43:59,...,NA/Swap OIS USD,Overnight Index Swap (OIS),Bermudan-Chooser,Elect at Exercise,QZPB5VSBGRCD,USD,OPTL,BERM,Vanilla,OPTL
35,35,1,Rates,Option,Swaption,UPI,QZJBL64PCV75,New,NaN,2024-10-28T21:10:58,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Chooser,Elect at Exercise,QZPB5VSBGRCD,USD,OPTL,EURO,Vanilla,OPTL
268,268,1,Rates,Option,Swaption,UPI,QZDV756RN422,New,NaN,2024-08-14T06:41:27,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZPB5VSBGRCD,USD,CALL,EURO,Other,PHYS
1139,1139,1,Rates,Option,Swaption,UPI,QZNK0LRWFJZC,New,NaN,2024-01-28T13:25:16,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Put,Physical,QZPB5VSBGRCD,USD,PUTO,EURO,Other,PHYS
1313,1313,1,Rates,Option,Swaption,UPI,QZX5X9WGZLCF,New,NaN,2024-01-11T09:22:57,...,NA/Swap OIS USD,Overnight Index Swap (OIS),American-Put,Physical,QZPB5VSBGRCD,USD,PUTO,AMER,Other,PHYS
3347,3347,1,Rates,Option,Swaption,UPI,QZBZRQ5RB4DG,New,NaN,2023-11-17T13:26:16,...,NA/Swap OIS USD,Overnight Index Swap (OIS),Bermudan-Put,Cash,QZPB5VSBGRCD,USD,PUTO,BERM,Vanilla,CASH
3656,3656,1,Rates,Option,Swaption,UPI,QZQ5WPTWF6W4,New,NaN,2023-11-17T07:46:59,...,NA/Swap OIS USD,Overnight Index Swap (OIS),American-Call,Cash,QZPB5VSBGRCD,USD,CALL,AMER,Vanilla,CASH
4016,4016,1,Rates,Option,Swaption,UPI,QZTKVQK1S1K1,New,NaN,2023-11-17T07:07:52,...,NA/Swap OIS USD,Overnight Index Swap (OIS),Bermudan-Call,Cash,QZPB5VSBGRCD,USD,CALL,BERM,Vanilla,CASH
4017,4017,1,Rates,Option,Swaption,UPI,QZPF9RBTP7JV,New,NaN,2023-11-17T07:07:48,...,NA/Swap OIS USD,Overnight Index Swap (OIS),American-Put,Cash,QZPB5VSBGRCD,USD,PUTO,AMER,Vanilla,CASH
5190,5190,1,Rates,Option,Swaption,UPI,QZWRZ8NRF2MW,New,NaN,2023-11-10T15:52:51,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Chooser,Cash,QZPB5VSBGRCD,USD,OPTL,EURO,Vanilla,CASH


In [ ]:
sdr_df